In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gzip

In [2]:
def sigmoid(x):
    return 1 / (1+ np.exp(-x))

def softmax(a):
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    
    return exp_a / sum_exp_a

### MNIST数据集

<img src="./mnist数据集样例.png" width="30%">

In [3]:
img_size = 28*28  #图像大小

dataset_dir = './data'   #MNIST数据集的保存路径

key_file = {
    'train_img':'train-images-idx3-ubyte.gz',    #训练数据
    'train_label':'train-labels-idx1-ubyte.gz',  #训练数据标签
    'test_img':'t10k-images-idx3-ubyte.gz',      #测试数据
    'test_label':'t10k-labels-idx1-ubyte.gz'     #测试数据预测值标签
}

def _load_img(file_name):    #加载图像数据
    file_path =  dataset_dir + "/" + file_name
   
    with gzip.open(file_path, 'rb') as f:
        data = np.frombuffer(f.read(), np.uint8, offset=16)
    data = data.reshape(-1, img_size)
    
    return data


def _load_label(file_name):  #加载标签数据
    file_path = dataset_dir + "/" + file_name
    
    with gzip.open(file_path, 'rb') as f:
        labels = np.frombuffer(f.read(), np.uint8, offset=8)
    
    return labels

def get_data():
    x_train = _load_img(key_file['train_img'])
    t_train = _load_label(key_file['train_label'])
    x_test = _load_img(key_file['test_img'])
    t_test = _load_label(key_file['test_label'])
    
    return x_train, t_train, x_test, t_test

In [4]:
x_train, t_train, x_test, t_test = get_data()

In [5]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('t_train shape:', t_train.shape)

x_train shape: (60000, 784)
x_test shape: (10000, 784)
t_train shape: (60000,)


In [6]:
#图片显示

img = x_train[0].reshape(28, 28)

from PIL import Image
pil_img = Image.fromarray(np.uint8(img))    #实现array到image的转换
pil_img.show()

In [7]:
t_train[0]

5

### 构建神经网络模型

这里假设神经网络的参数已经确定(关于如何确定神经网络的参数，后面会讲到)<br>
通过这个数据集，演示网络的前向传播过程

In [8]:
## W,b 

import pickle

def init_network():
    with open("sample_weight.pkl", "rb") as f:
        network = pickle.load(f)
    
    return network

In [9]:
network = init_network()

In [10]:
network.keys()

dict_keys(['b2', 'W1', 'b1', 'W2', 'W3', 'b3'])

In [11]:
print(network["W1"].shape)
print(network["b1"].shape)
print(network["W2"].shape)
print(network["b2"].shape)
print(network["W3"].shape)
print(network["b3"].shape)

(784, 50)
(50,)
(50, 100)
(100,)
(100, 10)
(10,)


In [12]:
#构建的是一个3层的神经网络模型，输入是784维，第一个隐藏层是50个节点，第二个隐藏层有100个节点，输出层有10个节点

In [13]:
def predict(network, x):
    W1, W2, W3 = network['W1'],network['W2'],network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    
    z1 = np.dot(x, W1) + b1
    a1 = sigmoid(z1)
    z2 = np.dot(a1, W2) + b2
    a2 = sigmoid(z2)
    z3 = np.dot(a2, W3) + b3
    y = softmax(z3)
    
    return y

In [14]:
##用test数据进行预测，并label标签进行比对，获取模型的精度、准确度
accuracy = 0

for i in range(len(x_test)):
    y_hat = predict(network, x_test[i])
    #y_hat 与 t_test[i]进行比对
    p = np.argmax(y_hat)
    if p == t_test[i]:
        accuracy += 1
        
print("Accuracy:", float(accuracy) / len(x_test))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


Accuracy: 0.9207


In [15]:
y_batch_hat = np.array([
    [0.1, 0.2, 0.8, 0.01, 0.01],
    [0.3, 0.4, 0.5, 0.01, 0.01],
    [0.8, 0.01, 0.01, 0.01, 0.01]
])
np.argmax(y_batch_hat, axis=1)

array([2, 2, 0], dtype=int64)

### 批处理batch size

In [16]:
batch_size = 100 #批数量

accuracy_cnt = 0

for i in range(0, len(x_test), batch_size):
    x_batch = x_test[i: i+batch_size]
    y_batch_hat = predict(network, x_batch)
    #y_batch_hat 与 t_test[i:i+batch_size]进行比对
    p = np.argmax(y_batch_hat, axis=1)
    accuracy_cnt += np.sum(p==t_test[i:i+batch_size])
        
print("Accuracy:", float(accuracy_cnt) / len(x_test))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


Accuracy: 0.9207
